# Logbook Metagenomics Project Salinity Stress
**abstract**
**Introduction**

## Logs
**Date: 5-03-2024**
**Goals**
- Seek tools needed for processing and analysing our data

**Results**
I was able to find a wikipedia page giving a nive overview of all the steps involved in a metagenomics analysis. [metagenomics](https://en.wikipedia.org/wiki/Metagenomics). For us the main interest is in the non-assambled analysis. Because unreliable reads should not be assambled and we don't want homunculi genomes that we then try to find the organisms for. We want to use every read as a reference to find our potential organisms. This is called classification for which i have found atleast 

This was a good reference project [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC8086013/)
I also found nanoclass2 which can taxonomic classify using a bunch of tools, though it seems this is tool does everything for you, which might not be something for us [here](https://ndombrowski.github.io/NanoClass2/source/run_nanoclass.html)

MegaBLAST, Centrifuge, Kraken2,RDP  ??> IDTAXA, [SPINGO](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0747-1)

https://github.com/lsbnb/MetaSquare


A big question is which classifiers should we use to guarentee that the classified organisms are accurate. Because we don't know what organisms to expect and want to do a broad as possible search, a combination of these tools would be good choice.

Porechop for trimming adapters 
Filtlong or nanofilt for filtering reads based on quality.
Centrifuge, Kraken2, RDP and MegaBLAST for Taxonimy classification.


**Refelction**
There is a large variety of tools and directions a metagenommics project can go. I am having a hard time creating something concreate, Ronald is hinting, sortof, but i am only more confused on what would be the best direction for our project. I know for sure classifing will get us a good way. another potential direction might be comparitive metagenomics.

**Next Steps**
- Testing Trimming tools 